# Experiment 05-01

In [ ]:
import numpy as np
import hashlib
from binascii import hexlify
import pyopencl as cl
from Library.opencl_information import opencl_information

## Show the available Platforms

In [ ]:
info = opencl_information()
info.print_full_info()

## Configure the OpenCL Context

In [ ]:
platform_number = 0
device_number = 0

cl_devices = cl.get_platforms()[platform_number].get_devices()
cl_ctx = cl.Context(cl_devices)
cl_queue = cl.CommandQueue(cl_ctx, cl_devices[device_number])

## Compile the Program

In [ ]:
def build_program(program_files : list, cl_ctx : cl.Context,
        build_options=[]) -> cl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    cl_ctx : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source = ''

    for cl_file in program_files:
        with open(cl_file, 'r') as cl_file:
            file_source = cl_file.read()
            program_source += '\n' + file_source

    program_source = cl.Program(cl_ctx, program_source)
    program = program_source.build(options=build_options)
            
    return program

In [ ]:
cl_program_files = [
    'Library/worker/sha256.cl',
    'Library/worker/zimcoin.cl',
]

cl_program = build_program(cl_program_files, cl_ctx)

# show the kernel names
program_kernel_names = cl_program.get_info(cl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

## Initial Tests

### Single Hash

In [ ]:
# the plaintext to hash
#plaintext = 'bells hel the quick brown fox jumps over the lazy dog 12356'
plaintext = '665782'
#plaintext = 'this is a description of the latest block' + '395707976'
#plaintext = '120Make America Great Again'
#plaintext = '69817Make America Great Again'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

# the hash output
hash_output = np.zeros(8, dtype=np.uint32)
leading_zeros = np.zeros(1, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_plaintext_bytes = cl.Buffer(
    cl_ctx,
    cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,
    hostbuf=plaintext_bytes)

cl_plaintext_length = cl.Buffer(
    cl_ctx,
    cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR,
    hostbuf=plaintext_length)

cl_hash_output = cl.Buffer(
    cl_ctx,
    cl.mem_flags.WRITE_ONLY,
    hash_output.nbytes)

cl_leading_zeros = cl.Buffer(
    cl_ctx,
    cl.mem_flags.WRITE_ONLY,
    leading_zeros.nbytes)

# execute the program
cl_program.get_single_hash(
    cl_queue, (1,), None,
    cl_plaintext_bytes,
    cl_plaintext_length,
    cl_hash_output,
    cl_leading_zeros)

# get the results
cl.enqueue_copy(cl_queue, hash_output, cl_hash_output)
cl.enqueue_copy(cl_queue, leading_zeros, cl_leading_zeros)

# print the results
print("Plaintext: %s" % plaintext)
print("Leading Zeros: %d" % leading_zeros[0])
print("CL result      : %s" % hexlify(hash_output))
print("Correct result : %s" % hexlify(hashlib.sha256(plaintext.encode('utf-8')).digest()))
print("CL output      : %s" % hash_output)
print("CL output      : %s" % np.frombuffer(hash_output, dtype=np.uint8))
print("CL output      : %s" % hash_output.tobytes())

assert hexlify(hash_output) == hexlify(hashlib.sha256(plaintext.encode('utf-8')).digest())


### Random String Generation

In [ ]:
# set up the variables to generate the random numbers
seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
start = np.uint8(32)
end = np.uint8(126)
length = np.uint32(4)
result = np.zeros(length, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_start = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=start)
cl_end = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=end)
cl_len = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=length)
cl_result = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, result.nbytes)

# execute the program
cl_program.get_random_numbers(
    cl_queue, (1,), None,
    cl_seed,
    cl_start,
    cl_end,
    cl_len,
    cl_result)

# get the results
cl.enqueue_copy(cl_queue, result, cl_result)

# print the results
print(f"Seed: {seed}")
print(f"Result: {result}")
print(f"Result String: {result.tobytes()}")

In [ ]:
# set up the variables to generate the random numbers
seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
result = np.zeros(shape=16, dtype=np.uint8)
length = np.zeros(shape=(1,), dtype=np.uint8)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_result = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, result.nbytes)
cl_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, length.nbytes)

# execute the program
cl_program.get_random_string(
    cl_queue, (1,), None,
    cl_seed,
    cl_result,
    cl_len)

# get the results
cl.enqueue_copy(cl_queue, length, cl_len)
cl.enqueue_copy(cl_queue, result, cl_result)

# print the results
print(f"Length: {length}")
print(f"Result: {result}")
print(f"Result String: {result.tobytes()}")

## Test get_single_hash_nonce

In [ ]:
# set up the variables to generate the random numbers
plaintext = 'this is a description of the latest block'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
nonce = np.zeros(shape=16, dtype=np.uint8)
nonce_len = np.zeros(shape=1, dtype=np.uint8)
hash = np.zeros(8, dtype=np.uint32)

# allocate the memory for the variables on the device
cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)
cl_plaintext_bytes = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_bytes)
cl_plaintext_length = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_length)
cl_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_nonce_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce_len.nbytes)
cl_hash_output = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, hash_output.nbytes)

# execute the program
cl_program.get_single_hash_nonce(
    cl_queue, (1,), None,
    cl_seed,
    cl_plaintext_bytes,
    cl_plaintext_length,
    cl_nonce,
    cl_nonce_len,
    cl_hash_output)

# get the results
cl.enqueue_copy(cl_queue, nonce, cl_nonce)
cl.enqueue_copy(cl_queue, nonce_len, cl_nonce_len)
cl.enqueue_copy(cl_queue, hash_output, cl_hash_output)

nonce_str = nonce[:nonce_len[0]].tobytes().decode('UTF-8')

# print the results
print("Seed: %s" % seed)
print("Plaintext: %s" % plaintext)
print("Nonce: %s" % nonce)
print("Nonce Length: %s" % nonce_len[0])
print("Nonce String: %s" % nonce_str)
print("Hash Output: %s" % hash_output)
print("Hash Output string: %s" % hexlify(hash_output))
#print("No Nonce          : %s" % hexlify(hashlib.sha256((plaintext).encode('utf-8')).digest()))
print("Correct result    : %s" % hexlify(hashlib.sha256((plaintext+nonce_str).encode('utf-8')).digest()))

assert hexlify(hash_output) == hexlify(hashlib.sha256((plaintext+nonce_str).encode('utf-8')).digest())


## Mine Nonce

In [ ]:
# set up the variables to generate the random numbers
plaintext = 'this is a description of the latest block'
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))

seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
window_size = np.uint32(1000000)
nonce = np.zeros(shape=16 * 64, dtype=np.uint8)
nonce_len = np.zeros(shape=64, dtype=np.uint8)

# allocate the memory for the variables on the device
cl_window_size = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=window_size)
cl_plaintext_bytes = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_bytes)
cl_plaintext_length = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_length)
cl_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_nonce_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce_len.nbytes)

zeros_found = {}

while (True):
    seed = np.random.randint(0, np.iinfo(np.uint32).max, dtype=np.uint32)
    cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)

    # execute the program
    cl_program.mine_nonce(
        cl_queue, (1,), None,
        cl_seed,
        cl_window_size,
        cl_plaintext_bytes,
        cl_plaintext_length,
        cl_nonce,
        cl_nonce_len)

    # get the results
    cl.enqueue_copy(cl_queue, nonce, cl_nonce)
    cl.enqueue_copy(cl_queue, nonce_len, cl_nonce_len)

    # interpret the results
    for i in range(0, 64):
        if nonce_len[i] > 0:
            if (i not in zeros_found):
                nonce_str = nonce[i * 16:i * 16 + nonce_len[i]].tobytes().decode('UTF-8')
                zeros_found[i] = nonce_str
                hash = hashlib.sha256((plaintext + nonce_str).encode('utf-8'))

                print("%4d: [%2d] %16s %64s" % (i, nonce_len[i], nonce_str, hash.hexdigest() if nonce_len[i] > 0 else ''))